In [2]:
# import libraries
import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\Users\harip\Desktop\Data Analytics Project\Advanced Analytics & Dashboard Design\Exercise 6.1'

In [4]:
# import excel file
data = pd.read_csv(os.path.join(path,'02 Data','Original Data','flavors_of_cacao.csv'), index_col = False)

In [5]:
# quick check
data.head()

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


# Renaming Columns

In [6]:
# Renaming columns
# First, strip whitespace and remove newlines
data.columns = data.columns.str.replace(r'\s+', ' ', regex=True).str.strip()

# Now rename specific columns
rename_dict = {
    'Company (Maker-if known)': 'Company',
    'Specific Bean Origin or Bar Name': 'Specific Bean Origin',
    'Review Date': 'Review',
    'Cocoa Percent': 'Cacao percent',
    'Company Location': 'Location',
    'Bean Type': 'Bean type',
    'Broad Bean Origin': 'Broad Bean Origin'
}

data.rename(columns=rename_dict, inplace=True)

print(data.columns)

Index(['Company', 'Specific Bean Origin', 'REF', 'Review', 'Cacao percent',
       'Location', 'Rating', 'Bean type', 'Broad Bean Origin'],
      dtype='object')


In [7]:
data.head()

,Company,Specific Bean Origin,REF,Review,Cacao percent,Location,Rating,Bean type,Broad Bean Origin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


# Checking Mixed-typed Data

In [8]:
# Check Mixed type Data
for col in data.columns.tolist():
  weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (data[weird]) > 0:
    print (col)

Bean type
Broad Bean Origin


C:\Users\harip\AppData\Local\Temp\ipykernel_31264\3833885844.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\harip\AppData\Local\Temp\ipykernel_31264\3833885844.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\harip\AppData\Local\Temp\ipykernel_31264\3833885844.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\harip\AppData\Local\Temp\ipykernel_31264\3833885844.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\harip\AppData\Local\Temp\ipykernel_31264\38

In [9]:
# check data types
print(data.dtypes)

Company                  object
Specific Bean Origin     object
REF                       int64
Review                    int64
Cacao percent            object
Location                 object
Rating                  float64
Bean type                object
Broad Bean Origin        object
dtype: object


In [10]:
# Clean Bean type and Broad Bean Origin → string dtype, keep NaN
data['Bean type'] = data['Bean type'].astype('string')
data['Broad Bean Origin'] = data['Broad Bean Origin'].astype('string')

# Clean Cacao percent → numeric for analysis
data['Cacao percent'] = (
    data['Cacao percent']
    .str.replace('%', '', regex=False)
    .str.strip()
    .astype(float)
    .round()
    .astype('Int64')  # nullable integer type
)

In [11]:
# check data types
print(data.dtypes)

Company                         object
Specific Bean Origin            object
REF                              int64
Review                           int64
Cacao percent                    Int64
Location                        object
Rating                         float64
Bean type               string[python]
Broad Bean Origin       string[python]
dtype: object


# Find Missing values

In [12]:
# Find Missing Data
data.isnull().sum()

Company                 0
Specific Bean Origin    0
REF                     0
Review                  0
Cacao percent           0
Location                0
Rating                  0
Bean type               1
Broad Bean Origin       1
dtype: int64

In [13]:
# Count blank/space-only cells per column
# Include both object and string types
blank_counts = data.select_dtypes(include=['object', 'string']).apply(lambda x: x.str.strip().eq('').sum())
print(blank_counts)

Company                   0
Specific Bean Origin      0
Location                  0
Bean type               887
Broad Bean Origin        73
dtype: int64


In [14]:
# Replace blanks or spaces in 'Bean type' and 'Broad Bean Origin' with 'Unknown'
data['Bean type'] = data['Bean type'].replace(r'^\s*$', 'Unknown', regex=True)
data['Broad Bean Origin'] = data['Broad Bean Origin'].replace(r'^\s*$', 'Unknown', regex=True)

In [15]:
# Check result 
print(data['Bean type'].value_counts())
print(data['Broad Bean Origin'].value_counts())

Bean type
Unknown                     887
Trinitario                  419
Criollo                     153
Forastero                    87
Forastero (Nacional)         52
Blend                        41
Criollo, Trinitario          39
Forastero (Arriba)           37
Criollo (Porcelana)          10
Trinitario, Criollo           9
Forastero (Parazinho)         8
Forastero (Arriba) ASS        6
Beniano                       3
Matina                        3
Nacional (Arriba)             3
EET                           3
Criollo (Ocumare 61)          2
Amazon mix                    2
Trinitario, Forastero         2
Amazon, ICS                   2
Criollo (Amarru)              2
Criollo, Forastero            2
Nacional                      2
Forastero (Catongo)           2
Trinitario (85% Criollo)      2
Criollo, +                    1
Criollo (Ocumare 77)          1
Criollo (Ocumare 67)          1
Criollo (Wild)                1
Criollo (Ocumare)             1
CCN51                         

# Check Duplicates

In [16]:
data_dups = data[data.duplicated()]

In [17]:
data_dups

,Company,Specific Bean Origin,REF,Review,Cacao percent,Location,Rating,Bean type,Broad Bean Origin


No duplicates values were found

# Basic Descriptive Statistics

In [18]:
# Basic statistics for numeric columns
numeric_cols = ['REF','Review','Cacao percent','Rating']
data[numeric_cols].describe()

,REF,Review,Cacao percent,Rating
count,1795.000000,1795.000000,1795.0,1795.000000
mean,1035.904735,2012.325348,71.697493,3.185933
std,552.886365,2.927210,6.323706,0.478062
min,5.000000,2006.000000,42.0,1.000000
25%,576.000000,2010.000000,70.0,2.875000
50%,1069.000000,2013.000000,70.0,3.250000
75%,1502.000000,2015.000000,75.0,3.500000
max,1952.000000,2017.000000,100.0,5.000000


# Descriptive stats for categorical variables

In [19]:
categorical_cols = ['Company', 'Specific Bean Origin', 'Location', 'Bean type', 'Broad Bean Origin']

for col in categorical_cols:
    print(f"{col}:")
    print("Unique values:", data[col].nunique())
    print("Top 5 most frequent values:\n", data[col].value_counts().head())
    print()

Company:
Unique values: 416
Top 5 most frequent values:
 Company
Soma        47
Bonnat      27
Fresco      26
Pralus      25
A. Morin    23
Name: count, dtype: int64

Specific Bean Origin:
Unique values: 1039
Top 5 most frequent values:
 Specific Bean Origin
Madagascar            57
Peru                  45
Ecuador               42
Dominican Republic    37
Venezuela             21
Name: count, dtype: int64

Location:
Unique values: 60
Top 5 most frequent values:
 Location
U.S.A.    764
France    156
Canada    125
U.K.       96
Italy      63
Name: count, dtype: int64

Bean type:
Unique values: 41
Top 5 most frequent values:
 Bean type
Unknown                 887
Trinitario              419
Criollo                 153
Forastero                87
Forastero (Nacional)     52
Name: count, dtype: Int64

Broad Bean Origin:
Unique values: 100
Top 5 most frequent values:
 Broad Bean Origin
Venezuela             214
Ecuador               193
Peru                  165
Madagascar            145
Do

In [20]:
# Export data in csv
data.to_csv(os.path.join(path,'02 Data','Prepared Data','clean_data.csv'))